In [ ]:
import os
import numpy as np
import glob
from pathlib import Path
import re

for path in Path('src').rglob('*.c'):
    print(path.name)

for dataset in ['abalone', 'house_sales', 'analcatdata_supreme', 'Bike_Sharing_Demand']:
    before_mse_list, after_mse_list = [], []
    log_dir = f"log/regression_dem/{dataset}"

    before_p = re.compile('test_loss before adaptation')
    after_p = re.compile('test_loss after adaptation')

    for path in Path(log_dir).rglob('*.txt'):
        f = "".join(open(path, "r").readlines())
        before_mse = before_p.search(f)
        after_mse = after_p.search(f)

        before_mse_list.append(before_mse)
        after_mse_list.append(after_mse)
    before_mse_list = np.array(before_mse_list)
    after_mse_list = np.array(after_mse_list)

    before_mean = np.mean(before_mse_list)
    before_se = np.std(before_mse_list, ddof=1) / np.sqrt(len(before_mse_list))

    after_mean = np.mean(after_mse_list)
    after_se = np.std(after_mse_list, ddof=1) / np.sqrt(len(after_mse_list))

    print(f"dataset: {dataset}")
    print(f"before: {before_mean:.4f} ± {before_se:.4f}")
    print(f"after: {after_mean:.4f} ± {after_se:.4f}")

In [ ]:
import os
import lib
import numpy as np
import pandas as pd

DATASETS = [
    "abalone",
    "adult",
    "buddy",
    "california",
    "cardio",
    "churn2",
    "default",
    "diabetes",
    "fb-comments",
    "gesture",
    "higgs-small",
    "house",
    "insurance",
    "king",
    "miniboone",
    "wilt"
]

_REGRESSION = [
    "abalone",
    "california",
    "fb-comments",
    "house",
    "insurance",
    "king",
]


method2exp = {
    "real": "exp/{}/ddpm_cb_best/",
    "tab-ddpm": "exp/{}/ddpm_cb_best/",
    "smote": "exp/{}/smote/",
    "ctabgan+": "exp/{}/ctabgan-plus/",
    "ctabgan": "exp/{}/ctabgan/",
    "tvae": "exp/{}/tvae/"
}

eval_file = "eval_catboost.json"
show_std = False
df = pd.DataFrame(columns=["method"] + [_[:3].upper() for _ in DATASETS])

for algo in method2exp: 
    algo_res = []
    for ds in DATASETS:
        if not os.path.exists(os.path.join(method2exp[algo].format(ds), eval_file)):
            algo_res.append("--")
            continue
        metric = "r2" if ds in _REGRESSION else "f1"
        res_dict = lib.load_json(os.path.join(method2exp[algo].format(ds), eval_file))

        if algo == "real":
            res = f'{res_dict["real"]["test"][metric + "-mean"]:.4f}' 
            if show_std: res += f'+-{res_dict["real"]["test"][metric + "-std"]:.4f}'
        else:
            res = f'{res_dict["synthetic"]["test"][metric + "-mean"]:.4f}'
            if show_std: res += f'+-{res_dict["synthetic"]["test"][metric + "-std"]:.4f}'

        algo_res.append(res)
    df.loc[len(df)] = [algo] + algo_res